In [18]:
import cv2
from scipy import signal
import numpy as np
import math

try:
    
   
    video=cv2.VideoCapture(0)
  #  video=cv2.VideoCapture("http://192.168.0.100:8080/video")  # this for mobil connection cam
    hue,sat,val = 0,0,0 #initial values for hue,sat,val
    kernelErosion = np.ones((1,1),np.uint8) #kernel for erosion
    kernelDilation = np.ones((6,6),np.uint8) #kernel for dilation
    
    width = video.get(3) #width
    height = video.get(4) #height
    
    #these points to draw the test calibration color later
    points =[]
    x = 10
    y = 70
    points.append((x/100,y/100))
    
    points.append((x/100,(y - 20)/100))
    points.append((x/100,(y+10)/100))
    points.append(((x +5)/100,y/100))
    
    points.append(((x +5)/100,(y-15)/100))
    points.append(((x +5)/100,(y+10)/100))
    points.append(((x +10)/100,y/100))
    points.append(((x +10)/100,(y+5)/100))
    points.append(((x +10)/100,(y+10)/100))
    
    print(width,height)
    while True:
        frame = getFrame() #get frame from running video
        frame = cv2.blur(frame, (10,10)) # blur the original frame "very important for the final image"
       # frame = cv2.GaussianBlur(frame,(15,15),0)
        hsv = convertToHsv(frame) #get hsv verison from the frame
        makeCalibrationRectangles(frame,10,10)# draw the calibration rectangles to take the hand color
        
        #get the average threshold for the colors of hsv
        #hue,sat,val = getTotalAvgHueSatVal(hsv,10,10)
        
        masks = getTotalAvgHueSatVal(hsv,10,10)
       #make a mask from lower to upper band of the colors
       #the lower and upper band  withing the function from inside
        #masks =mask(hsv,[hue,sat,val])
        
        #masks = cv2.medianBlur(masks,9)
        
        #make erison for the mask to eliminate noise "smaller white pixels not important"
        erosion = cv2.erode(masks,kernelErosion,iterations =1)
        
        #make dilation to close the black areas inside the hand after you removed the noise
        dilation = cv2.dilate(erosion,kernelDilation,iterations = 2)
        
        #dilation = cv2.blur(dilation, (3,3)) # blur the image #@deprecated
        #dilation = cv2.medianBlur(dilation, 3) #@deprecated

        #make edges
        #we found that this is not much important
        #edge = cv2.Canny(dilation, 175, 175) #@deprecated
    
        #get the contour for all the white areas in the frame
        _, contours, _ = cv2.findContours(dilation,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
        #drawLargesContours(contours,frame,1000) #@deprecated
        
        #get the index of the largest contour
        index = getLargestCountour(contours)
        
        # if the index is -1 then there was no contours from the begining then move
        if index >=0 and index <= len(contours) -1:
            cnt = contours[index]
            
            #get the points of the parameter that shape the outer convex
            #false parameter means to retrieve with the hull the convexity  defects (the tie between two fingers)
            hull = cv2.convexHull(cnt,returnPoints = False)
            
            #get the ties
            defects = cv2.convexityDefects(cnt,hull)
            #draw the defects
            drawDefects(defects)
            
        frame2 = frame
        
       #segment the image
        segmented = cv2.bitwise_and(frame,frame2, mask= masks)
        
        #print all the screens you want
        imgsShow()     
        
        key=cv2.waitKey(1)
        if key == ord('q'):
            '''print(frame) '''  
            break
        if key == ord('w'):
            hand=frame[int(y/3):int(2*y/3),int(x/3):int(2*x/3)]
            hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
            cv2.imshow("image1",hsv)
except Exception as e:
    print(e)
video.release()




cv2.destroyAllWindows()
    

640.0 480.0


In [2]:
#get on frame
def getFrame():
     _,frame=video.read() #get the video frame by frame
     return frame

In [3]:
#hsv version
def convertToHsv(frame):
    return cv2.cvtColor(frame, cv2.COLOR_BGR2HSV) #convert to hsv space
    

In [4]:
#draw rectangles to take color hand
#x,y the width and height of the rectangle
def makeCalibrationRectangles(frame,x,y):
    for num in points:
            makeRectangle(frame,(int(num[0]*width),int(num[1]*height)),10,10) # draw rectangles 
    

In [5]:
#make rectangle on the frame
#leftUpperCorner: the point on the frame
#x,y : move from the point to x distance and y distance
def makeRectangle(frame,leftUpperCorner,x,y):
    rightLowerCorner =(leftUpperCorner[0] +x,leftUpperCorner[1] +y)
    cv2.rectangle(frame,leftUpperCorner,rightLowerCorner,(0,255,0),2) # draw rectangle


In [6]:
#return the hue,sat,val average values for single rectangle
#x width of the rectangle
#y height of the rectangle
def getAvgHueSatVal(fram,x0,y0,x,y):
    try:
        hue = 0.0
        sat = 0.0
        val = 0.0
        total = x*y
        lowX = x0
        lowY =y0
        highX = lowX + x
        highY = lowY + y      
        for i in range(lowX,highX):
            for j in range(lowY,highY):
                hue = hue + float(fram[i,j,0])/float(total)
                sat = sat + float(fram[i,j,1])/float(total)
                val = val + float(fram[i,j,2])/float(total)  

    except Exception as e:
        print(e)
    
    return hue,sat,val

In [15]:
#get the values for hue,sat,val from averaging all the rectangles
def getTotalAvgHueSatVal(fram,x,y):
    masks = []
    try:
        for rect in points:
            x0 = int(rect[0]*width)
            y0 = int(rect[1]*height)
            hue,sat,val = getAvgHueSatVal(fram,y0,x0,x,y)
            masks.append(mask(fram,[int(hue),int(sat),int(val)]))
            

        m1 = masks[0]
       # m1 = mask6Masks(masks)
        #m1 = cv2.addWeighted(masks[0], 1, masks[1], 1, 0.0)
    except Exception as e:
        print(e)
    return m1

In [16]:
#loop on all the masks and get the sum of them and return the mask
def mask6Masks(masks):
    try:
        m = masks[0].copy()
        for i in range(len(points) - 1):
            masks[i] = cv2.addWeighted(m, 1, masks[i+1], 1, 0.0)
            m = masks[i].copy()
    except Exception as e:
        print(e)                              
    return m               
            

In [9]:
#this will mask the image into binary and return the image
def mask(frame,bound):
   #this is the limit threshold the lowest from the average value and the highest from the average value
    low = 15
    high = 15
    try:
        lower_blue = np.array([bound[0] - low,bound[1] - 10 ,bound[2] - 10 ])
        upper_blue = np.array([bound[0] + high,bound[1] + 10 ,bound[2] + 10 ])
        
        mask = cv2.inRange(frame, lower_blue, upper_blue)
    except Exception as e:
        print(e)
        return frame
    
    

    return mask

In [10]:
#print all the screens you want
def imgsShow():
    cv2.imshow("image",frame)
    cv2.imshow("erosion",erosion)
    cv2.imshow('dilation',dilation)
    cv2.imshow('hsv',hsv)
    cv2.imshow('masks',masks)
    #cv2.imshow('segmented',segmented)

In [11]:
#return only  the index of the largest contour
def getLargestCountour(contours):
    largestArea = 0
    index = 0
    for i in range(len(contours)):
        area = cv2.contourArea(contours[i])
        if (area>largestArea):
                largestArea=area
                index= i
    return index       

In [12]:
#to print each tie according to the blue and red color and position
def printTies(vec):
    for obj in vec:
        start = obj[0]
        tie = obj[1]
        color = obj[2]
        if color == True: #red
              cv2.circle(frame,tie,5,[0,0,255],-1)
        else: #blue
             cv2.circle(frame,tie,5,[255,0,0],-1)
        cv2.line(frame,start,tie,[150,150,150],2)           
    vec.clear()   

In [13]:
#draw the defects
def drawDefects(defects):
    
            #put inside each element the tie x,y and the color   true==> red   &  false ==> blue  & start of the finger
            vec = []
            
            #blue refer to reverse hand
            #count number of blue ties
            countDefectsBlue = 0
            
            #red refer to straight hand
            #count number of red ties
            countDefectsRed = 0
            try:
                for i in range(defects.shape[0]):
                    
                    s,e,f,d = defects[i,0]
                    start = tuple(cnt[s][0]) #finger top
                    end = tuple(cnt[e][0]) #another finger top
                    tie = tuple(cnt[f][0]) # tie between them
                    
                    #if the distance between start line with end to the tie is less than 1000 this means that this not a tie (noise for example)
                    if d < 1000: 
                        continue
                        
                    cv2.line(frame,start,end,[0,255,0],2)  
                
                    # find length of all sides of triangle
                    a = math.sqrt((end[0] - start[0])**2 + (end[1] - start[1])**2) # distance between finger top and another finger top
                    b = math.sqrt((tie[0] - start[0])**2 + (tie[1] - start[1])**2) # distance between finger top and a tie
                    c = math.sqrt((end[0] - tie[0])**2 + (end[1] - tie[1])**2) # distance between another finger top and a tie
                    
                    # apply cosine rule here
                    angle = math.acos((b**2 + c**2 - a**2)/(2*b*c)) * 180/3.14

                    #if the angle is less than 50 then it's not a tie
                    if angle <= 50:
                        # if the tie y axis is less than y axis for top finger or the other one then  the hand is reversed
                        if tie[1] < start[1] or tie[1] <end[1]:
                            countDefectsBlue+= 1 # increment the blue defects
                            vec.append((start,tie,False)) # push the tie into the array to print later
                        elif tie[1] > start[1] or tie[1] > end[1]:
                            countDefectsRed += 1 # increment the red defects
                            vec.append((start,tie,True)) # push the tie into the array to print later
            except Exception as e:
                    print("hello")
                    print(e)
        
            #print the ties
            printTies(vec)          

In [14]:
#@deprecated
#draw contours that is higher than size
def drawLargesContours(countours,frame,size):
    for i in range(len(contours)):
        area = cv2.contourArea(contours[i])
        if (area>size):
                img = cv2.drawContours(frame, contours[i],-1 ,(0,255,0), 3) 

In [265]:
'''
@depercated


import cv2
from scipy import signal
import numpy as np

try:
    video=cv2.VideoCapture(0)
    hue,sat,val = 0,0,0
    while True:
        check,frame=video.read()
        blur = cv2.blur(frame, (3, 3)) # blur the image
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        x = video.get(3)
        y = video.get(4)
        cv2.rectangle(frame,(int(x/3),int(y/3)),(int(2*x/3),int(2*y/3)),(0,255,0),3)
        
        'take the hand area as an image '
        hand = hsv[int(y/3):int(2*y/3),int(x/3):int(2*x/3)]
        'get the average threshold for the colors of hsv'
        hue,sat,val = getAverageThreshold(hand)
        'make a mask from lower to upper band of the colors'
        masks =mask(hsv,[hue,sat,val])
        frame2 = frame
        'segment the image'
        segmented = cv2.bitwise_and(frame,frame2, mask= masks)
        'print all the screens you want'
        imgsShow()     
        
        key=cv2.waitKey(1)
        if key == ord('q'):
            print(frame)
            break
        if key == ord('w'):
            hand=frame[int(y/3):int(2*y/3),int(x/3):int(2*x/3)]
            hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
            cv2.imshow("image1",hsv)
except Exception as e:
    print(e)
video.release()




cv2.destroyAllWindows()

    '''

'\n@depercated\n\n\nimport cv2\nfrom scipy import signal\nimport numpy as np\n\ntry:\n    video=cv2.VideoCapture(0)\n    hue,sat,val = 0,0,0\n    while True:\n        check,frame=video.read()\n        blur = cv2.blur(frame, (3, 3)) # blur the image\n        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)\n        x = video.get(3)\n        y = video.get(4)\n        cv2.rectangle(frame,(int(x/3),int(y/3)),(int(2*x/3),int(2*y/3)),(0,255,0),3)\n        \n        \'take the hand area as an image \'\n        hand = hsv[int(y/3):int(2*y/3),int(x/3):int(2*x/3)]\n        \'get the average threshold for the colors of hsv\'\n        hue,sat,val = getAverageThreshold(hand)\n        \'make a mask from lower to upper band of the colors\'\n        masks =mask(hsv,[hue,sat,val])\n        frame2 = frame\n        \'segment the image\'\n        segmented = cv2.bitwise_and(frame,frame2, mask= masks)\n        \'print all the screens you want\'\n        imgsShow()     \n        \n        key=cv2.waitKey(1)\n  

In [50]:
#return the average for the three values of any image
'''
@deprecated
def getAverageThreshold(frame):
    try:
        x = frame.shape[0]
        y = frame.shape[1]
        hue = 0.0
        sat = 0.0
        val = 0.0
        total = 60*60
        lowx = int(x/2 -30)
        lowy = int(y/2 -30)
        highx = int(x/2 + 30)
        highy = int(y/2 + 30)
        
        for i in range(lowx,highx):
            for j in range(lowy,highy):
                hue = hue + float(frame[i,j,0])/float(total)
                sat = sat + float(frame[i,j,1])/float(total)
                val = val + float(frame[i,j,2])/float(total)
        hue = int(hue)
        sat = int(sat)        
        val = int(val)        

    except Exception as e:
        print(e)
    
    return hue,sat,val
'''
        

'\n\ndef getAverageThreshold(frame):\n    try:\n        x = frame.shape[0]\n        y = frame.shape[1]\n        hue = 0.0\n        sat = 0.0\n        val = 0.0\n        total = 60*60\n        lowx = int(x/2 -30)\n        lowy = int(y/2 -30)\n        highx = int(x/2 + 30)\n        highy = int(y/2 + 30)\n        \n        for i in range(lowx,highx):\n            for j in range(lowy,highy):\n                hue = hue + float(frame[i,j,0])/float(total)\n                sat = sat + float(frame[i,j,1])/float(total)\n                val = val + float(frame[i,j,2])/float(total)\n        hue = int(hue)\n        sat = int(sat)        \n        val = int(val)        \n\n    except Exception as e:\n        print(e)\n    \n    return hue,sat,val\n'

In [126]:
#get the values for hue,sat,val from averaging all the rectangles
''' 
def getTotalAvgHueSatVal(fram,x,y):
    try:
        hues,sats,vals = 0.0,0.0,0.0
        for rect in points:
            x0 = int(rect[0]*width)
            y0 = int(rect[1]*height)
            hue,sat,val = getAvgHueSatVal(fram,y0,x0,x,y)
            hues += hue
            sats += sat
            vals += val
            

        size = float(len(points))
      #  size = 1
        hues = int(hues / size)
        sats = int(sats / size)
        vals = int(vals / size)
        print(hues,sats,vals)
    except Exception as e:
        print(e)
    return hues,sats,vals
    '''

' \ndef getTotalAvgHueSatVal(fram,x,y):\n    try:\n        hues,sats,vals = 0.0,0.0,0.0\n        for rect in points:\n            x0 = int(rect[0]*width)\n            y0 = int(rect[1]*height)\n            hue,sat,val = getAvgHueSatVal(fram,y0,x0,x,y)\n            hues += hue\n            sats += sat\n            vals += val\n            \n\n        size = float(len(points))\n      #  size = 1\n        hues = int(hues / size)\n        sats = int(sats / size)\n        vals = int(vals / size)\n        print(hues,sats,vals)\n    except Exception as e:\n        print(e)\n    return hues,sats,vals\n    '